In [7]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [8]:
import os
AA_TOKEN=os.getenv("AA_TOKEN")
NAMESPACE=os.getenv("AA_NAMESPACE")

# Question & Answer

## SingleChunkQA

In [13]:
from typing import Sequence
from intelligence_layer.core import (
    Task,
    TextHighlight,
    CompleteInput,
    TaskSpan,
    NoOpTracer,
    TextHighlightInput, Llama3InstructModel,
)
from pydantic import BaseModel


class Input(BaseModel):
    chunk: str
    question: str


class Output(BaseModel):
    answer: str
    highlights: Sequence[str]


class SingleChunkQaTask(Task[Input, Output]):
    def __init__(
        self,
        model: Llama3InstructModel | None = None,
    ):
        super().__init__()
        self._model = model or Llama3InstructModel("pharia-1-llm-7b-control")

    def do_run(self, input: Input, task_span: TaskSpan) -> Output:
        instruction = f'Beantworte die Frage anhand des Textes. Wenn sich die Frage nicht mit dem Text beantworten lässt, antworte "Keine Antwort in den Quellen".\nQuellen: {input.chunk}\nFrage: {input.question}'
        prompt = self._model.to_instruct_prompt(instruction=instruction)
        input = CompleteInput(prompt=prompt)
        output = self._model.complete(input, task_span)

        highlight_task = TextHighlight(model=self._model)
        highlight_input = TextHighlightInput(
            rich_prompt=prompt,
            target=output.completion,
        )
        highlight_output = highlight_task.run(highlight_input, task_span)

        print(prompt.items)
        return Output(
            answer=output.completion,
            highlights=[
                prompt.items[0].text[highlight.start : highlight.end]
                for highlight in highlight_output.highlights
                if highlight.score > 0
            ],
        )


question = "Wer mag Pizza?"
chunk = "Tina mag nicht so gern Pizza. Mike aber mag Pizza sehr."

task = SingleChunkQaTask()
result = task.run(input=Input(chunk=chunk, question=question), tracer=NoOpTracer())
result

[Text(text='<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nBeantworte die Frage anhand des Textes. Wenn sich die Frage nicht mit dem Text beantworten lässt, antworte "Keine Antwort in den Quellen".\nQuellen: Tina mag nicht so gern Pizza. Mike aber mag Pizza sehr.\nFrage: Wer mag Pizza?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n', controls=[])]


Output(answer=' Mike mag Pizza.', highlights=['<|begin_of_text|><|start_header_id|>user<|end_header_id|>', 'Textes.', 'Pizza.', 'Mike', 'Pizza?<|eot_id|><|start_header_id|>assistant<|end_header_id|>'])

In [14]:
chunk
result.highlights

['<|begin_of_text|><|start_header_id|>user<|end_header_id|>',
 'Textes.',
 'Pizza.',
 'Mike',
 'Pizza?<|eot_id|><|start_header_id|>assistant<|end_header_id|>']

## MultiChunkRetrieverQA

In [18]:
collection_name = "rag-collection"
index_name = "rag-index1"
number_of_chunks = 5
query = "Was ist nohtyp?"

In [19]:
from intelligence_layer.connectors import (
    DocumentIndexClient,
    DocumentIndexRetriever,
)
from intelligence_layer.core import InMemoryTracer
from intelligence_layer.examples import MultipleChunkRetrieverQa, RetrieverBasedQaInput


document_index = DocumentIndexClient(
    token=AA_TOKEN,
)

document_index_retriever = DocumentIndexRetriever(
    document_index=document_index,
    index_name=index_name,
    namespace=NAMESPACE,
    collection=collection_name,
    k=5,
    threshold=0.5,
)

model = Llama3InstructModel("pharia-1-llm-7b-control")
retriever_qa = MultipleChunkRetrieverQa(
    document_index_retriever, 
    insert_chunk_number=number_of_chunks,
)

input = RetrieverBasedQaInput(
    question=query,
)
tracer = InMemoryTracer()

output = retriever_qa.run(input, tracer)
print("Answer: ", output.answer)
print("-"*50)
print("Information:")
print(output.model_dump_json(indent=2))

Answer:  nohtyp ist eine Programmiersprache, die entwickelt wurde, um Programmierer herauszufordern und zu unterhalten.
--------------------------------------------------
Information:
{
  "answer": "nohtyp ist eine Programmiersprache, die entwickelt wurde, um Programmierer herauszufordern und zu unterhalten.",
  "sources": [
    {
      "chunk": {
        "document_id": {
          "collection_path": {
            "namespace": "aleph-alpha",
            "collection": "rag-collection"
          },
          "document_name": "Erfunde-Programmiersprache.docx"
        },
        "chunk": "Einführung in Nohtyp\nWillkommen bei Nohtyp, der Programmiersprache, die alles auf den Kopf stellt! Nohtyp ist eine skurrile und einzigartige Sprache, die entwickelt wurde, um Programmierer herauszufordern und zu unterhalten. Hier sind die Grundlagen, um loszulegen.\n1. Syntax\nDie Syntax von Nohtyp ist ungewöhnlich und erfordert ein Umdenken. Hier sind einige grundlegende Regeln:\nRückwärts-Schreiben: Al

In [22]:
tracer.submit_to_trace_viewer()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tracer.submit_to_trace_viewer()                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /Users/jens.luecke/Library/Caches/pypoetry/virtualenvs/rag-qa-workshop-xsC4sNM2-py3.12/lib/pytho │
│ n3.12/site-packages/intelligence_layer/core/tracer/tracer.py:211 in submit_to_trace_viewer       │
│                                                                                                  │
│   208 │   │   ...                                                                                │
│   209 │                                                                                          │
│   210 │   def submit_to_trace_viewer(self) -> bool:                                              │
│ ❱ 211 │   │   return submit_to_trace_viewer(self.export_for_viewing())                           │
│   212                                                                                            │
│   213                                                                                            │
│   214 class ErrorValue(BaseModel):                                                               │
│                                                                                                  │
│ /Users/jens.luecke/Library/Caches/pypoetry/virtualenvs/rag-qa-workshop-xsC4sNM2-py3.12/lib/pytho │
│ n3.12/site-packages/intelligence_layer/core/tracer/in_memory_tracer.py:88 in export_for_viewing  │
│                                                                                                  │
│    85 │   │   │   │   │   "Found a log outside of a span. Logs can only be part of a span."      │
│    86 │   │   │   │   )                                                                          │
│    87 │   │   │   else:                                                                          │
│ ❱  88 │   │   │   │   exported_root_spans.extend(entry.export_for_viewing())                     │
│    89 │   │   return exported_root_spans                                                         │
│    90                                                                                            │
│    91                                                                                            │
│                                                                                                  │
│ /Users/jens.luecke/Library/Caches/pypoetry/virtualenvs/rag-qa-workshop-xsC4sNM2-py3.12/lib/pytho │
│ n3.12/site-packages/intelligence_layer/core/tracer/in_memory_tracer.py:171 in export_for_viewing │
│                                                                                                  │
│   168 │   │   │   if isinstance(entry, LogEntry):                                                │
│   169 │   │   │   │   logs.append(entry)                                                         │
│   170 │   │   │   else:                                                                          │
│ ❱ 171 │   │   │   │   exported_spans.extend(entry.export_for_viewing())                          │
│   172 │   │   exported_spans.append(                                                             │
│   173 │   │   │   ExportedSpan(                                                                  │
│   174 │   │   │   │   context=self.context,                                                      │
│                                                                                                  │
│ /Users/jens.luecke/Library/Caches/pypoetry/virtualenvs/rag-